In [ ]:
!pip install chardet
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 122.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import folium 
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

In [ ]:

import pandas as pd

link = "https://raw.githubusercontent.com/alexanderquispe/Diplomado_PUCP/main/_data/data_dengue_peru.csv"
data = pd.read_csv(link)
data.head(3)

<ipython-input-21-32c2c2622941>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(link)


,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN


In [ ]:
#Check type value
data.dtypes

Año                 int64
Semana              int64
Eventos o daños    object
Departamento       object
Distrito           object
Provincia          object
Ubigeo              int64
Casos              object
dtype: object

2.-Generate ubigeo for Departments and Provinces taking the first two and four numbers.


In [ ]:
data["Ubigeo"].isna().sum()

0

In [ ]:
data["UBIGEO"]=data["Ubigeo"].apply(lambda x:str(x))
def complete_ubigeo(ubigeo):
    if len(ubigeo)<6:
        ubigeo = "0"+ubigeo
    return ubigeo
data["UBIGEO"]=data["UBIGEO"].apply(complete_ubigeo)


In [ ]:

data["CCDD"]=data["UBIGEO"].apply(lambda x : x[:2])
data["IDPROV"]=data["UBIGEO"].apply(lambda x : x[:4])

In [ ]:
data.head(3)

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,UBIGEO,CCDD,IDPROV
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0,080914,08,0809
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,180101,18,1801
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN,180101,18,1801


In [ ]:

data["Type"]=data["Casos"].apply(lambda x : type(x))
data["Type"].value_counts()


<class 'float'>    107328
<class 'str'>       64816
Name: Type, dtype: int64

In [ ]:
data[data["Casos"]=="1,364"]

,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos,UBIGEO,CCDD,IDPROV,Type
76223,2017,15,Dengue,PIURA,PIURA,PIURA,200101,"1,364",200101,20,2001,<class 'str'>


In [ ]:

import re
def normaliza(text):
    if type(text)==str:
        text = re.sub(r'(\d+),(\d+)', r'\1\2', text)
        text=float(text)
    return text


In [ ]:
data["Casos_num"]=data["Casos"].apply(normaliza)

3.- Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values.


In [ ]:
!pip install mplleaflet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mplleaflet: filename=mplleaflet-0.0.5-py3-none-any.whl size=28581 sha256=e58aaadcb25311097a6781cbfc85e82d875b4d7ae8be2ada36b9c6a7658a6de3
  Stored in directory: /root/.cache/pip/wheels/54/11/d6/5d25309bb8b662bd29880f7ffec893b66ffa677086a436c041
Successfully built mplleaflet


In [ ]:

data2021=data[data["Año"]==2021]
import geopandas as gpd
import matplotlib.pyplot as plt
import mplleaflet
path_shp = "../Diplomado_PUCP/_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp"
districts_shp = gpd.read_file(path_shp)


In [ ]:
districts_shp.head(3)

In [ ]:
merged = districts_shp.merge(data2021[["Casos_num","UBIGEO"]],on="UBIGEO")

In [ ]:
merged.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

In [ ]:
merged.plot(column='Casos_num', cmap='Greens', ax=ax, legend=True, legend_kwds={'label': 'Number of Cases'})

In [ ]:
# set the title and axis labels
ax.set_title('Number of DENGUE Cases 2021 by District')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')


In [ ]:

# add color for NA values
ax.set_facecolor('lightgrey')

5.- Use geopandas to plot the number of cases by the department for all the years using subplots. Every subplot for each year. Do not forget to indicate the color of NA values. Use this shapefile. For this task, you will have to aggregate shapefiles at the department level.


In [ ]:
path_shp2 = "../Diplomado_PUCP/_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp"
departments_shp = gpd.read_file(path_shp2)

In [ ]:
departments_shp.head(2)

In [ ]:
# Aggregate the data by department and year
agg_data = data.groupby(["Año", "Departamento"])["Casos_num"].sum().reset_index()


In [ ]:
#Loop over years
years = sorted(data.Año.unique())
ncols = 3
nrows = (len(years)+ncols-1)//ncols

In [ ]:
#creating subplots

fig, axs = plt.subplots(nrows, ncols, figsize=(15, 15))

In [ ]:
# Loop over years
for i, year in enumerate(years):
    # Filter data for year
    year_data = agg_data[agg_data['Año'] == year]
    

In [ ]:
# Merge with shapefile
    merged = departments_shp.merge(year_data, left_on='NOMBDEP', right_on='Departamento')

In [ ]:
# Plot
    ax = axs[i // ncols, i % ncols]
    
    merged.plot(column='Casos_num', cmap='Blues', ax=ax, legend=True, legend_kwds={'label': 'Number of Cases'})
    ax.set_title(year)
    ax.axis('off')

In [ ]:
# add color for NA values
    ax.set_facecolor('red')

In [ ]:
for j in range(i+1, nrows*ncols):
    axs[j // ncols, j % ncols].set_visible(False)
    
plt.tight_layout()
plt.show()